In [1]:
## Sensitivity Analysis for Phenolic-Coal Crosslinking Project
# The aim is to attempt using the four settings below for sensitivity analysis:
        # Factor prioritisation** - which parameters are most influential?
        # Factor fixing** - which parameters can we ignore?
        # Factor mapping - which inputs matter for just this space in the model output?
        # (Metamodelling) - which parameters can I use to model my model?

# Questions to answer, Can we determine;
        # 1. What is the sensitivity of the model to the input parameters?
        # 2. Which parameters are most influential?
        # 3. Which parameters can be ignored?
        # 4. Which parameters can be mapped to a reduced space?
        # 5. Which parameters can be used to model the model?

# The aim is to attempt these 2 approaches:
    # Local approaches
    # Global approaches

In [22]:
## Heat of reaction for the total crosslinking reaction for the different rxns -- Experimental data

# 100% phenolic curing reaction
Hrxn_total = 31620 # Total Heat of reaction (100% phenolic) J/mol (from DSC)
Na = 6.022e23 # Avogadro's number 

# Hrxn for a set of phenol-phenol (Hrxn_pp) -- Hrxn_total/Na (J/ph_mol)
Hrxn_pp = Hrxn_total/Na # J/ph_mol
print("Hrxn per phenol-phenol reaction = ", Hrxn_pp)

# Sum of total H_rxn for all phenol-phenol reactions (Hrxn_total_pp)
# Hrxn_total_pp = Hrxn_pp * Na (J/mol)

# Based on the number of phenol molecules in each reaction the heat of reaction for each reaction can be calculated
# n_phenol = mass_phenol/molar_mass_novolac_phenol
mass_phenol = 0.008 # g
molar_mass_novolac_phenol = 600 # g/mol
n_molphen = mass_phenol/molar_mass_novolac_phenol # mol
print("n_molphen = ", n_molphen)

# How many phenol molecules per n_molphen of novolac phenol?
# n_phenol = n_molphen * Avogadro's number
n_phenol = n_molphen * Na # mol
print("n_phenol = ", n_phenol)

# The total Heat of reaction (phenol-phenol links) from simulation 
Hrxn_total_pp = Hrxn_pp * n_phenol # J/mol
print("Hrxn_total_pp = ", Hrxn_total_pp)

# Check with the experimental data
# Hrxn_total_exp = 31620 # J/mol
Hrxn_total_exp = Hrxn_total * n_molphen
print("Hrxn_total_exp = ", Hrxn_total_exp)





Hrxn per phenol-phenol reaction =  5.250747260046496e-20
n_molphen =  1.3333333333333333e-05
n_phenol =  8.029333333333334e+18
Hrxn_total_pp =  0.42160000000000003
Hrxn_total_exp =  0.4216


In [ ]:
# Input n_molphenol range of values 
# n_molphenol = np.linspace(0, 1, 100)
n_molphen_values = [1.33e-5, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

#Once the n_molphenol is picked, then n_phenol is calculated and the Hrxn_sim is calculated

In [33]:
def calculate_heat_of_reaction(n_molphen_values):
    # Constants
    Hrxn_total = 31620  # Total Heat of reaction (100% phenolic) J/mol (from DSC)
    Na = 6.022e23  # Avogadro's number

    results = []

    for n_molphen in n_molphen_values:
        # Calculate Hrxn for a phenol-phenol reaction (Hrxn_pp)
        Hrxn_pp = Hrxn_total / Na  # J/ph_mol

        # Calculate the total number of phenol molecules (n_phenol)
        n_phenol = n_molphen * Na  # mol

        # Calculate the total Heat of reaction for all phenol-phenol reactions
        Hrxn_total_pp = Hrxn_pp * n_phenol  # J/mol

        # Check with experimental data
        Hrxn_total_exp = Hrxn_total * n_molphen

        results.append((n_molphen, Hrxn_total_exp, Hrxn_total_pp))

    return results

# Example usage
n_molphen_values = [1.33e-5, 50, 100, 150, 200, 250, 300]
n_molphen_simulated_links = [1.33e-5, 30, 80, 120, 180, 210, 260]
experimental_results = calculate_heat_of_reaction(n_molphen_values)
simulated_results = calculate_heat_of_reaction(n_molphen_simulated_links)

# Print the experimental results
print("Results based on the crosslinking of the total phenolic resins present in the system:")
for n_molphen, Hrxn_total_exp, Hrxn_total_pp in experimental_results:
    print(f"n_molphen = {n_molphen:.2e} mols, Hrxn_total_exp = {Hrxn_total_exp:.2f} J, Hrxn_total_pp = {Hrxn_total_pp:.2f} J")

# Print the simulated results
print("\nResults based on the actual number of phenol-phenol crosslinks formed in the system:")
for result in simulated_results:
    n_molphen_sim, Hrxn_total_exp, Hrxn_total_pp = result
    print(f"n_molphen_sim = {n_molphen_sim:.2e} mols, Hrxn_total_exp = {Hrxn_total_exp:.2f} J, Hrxn_total_pp = {Hrxn_total_pp:.2f} J")


# Plot the results
# Create a figure
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)


Results based on the crosslinking of the total phenolic resins present in the system:
n_molphen = 1.33e-05 mols, Hrxn_total_exp = 0.42 J, Hrxn_total_pp = 0.42 J
n_molphen = 5.00e+01 mols, Hrxn_total_exp = 1581000.00 J, Hrxn_total_pp = 1581000.00 J
n_molphen = 1.00e+02 mols, Hrxn_total_exp = 3162000.00 J, Hrxn_total_pp = 3162000.00 J
n_molphen = 1.50e+02 mols, Hrxn_total_exp = 4743000.00 J, Hrxn_total_pp = 4743000.00 J
n_molphen = 2.00e+02 mols, Hrxn_total_exp = 6324000.00 J, Hrxn_total_pp = 6324000.00 J
n_molphen = 2.50e+02 mols, Hrxn_total_exp = 7905000.00 J, Hrxn_total_pp = 7905000.00 J
n_molphen = 3.00e+02 mols, Hrxn_total_exp = 9486000.00 J, Hrxn_total_pp = 9486000.00 J
n_molphen = 3.50e+02 mols, Hrxn_total_exp = 11067000.00 J, Hrxn_total_pp = 11067000.00 J
n_molphen = 4.00e+02 mols, Hrxn_total_exp = 12648000.00 J, Hrxn_total_pp = 12648000.00 J
n_molphen = 4.50e+02 mols, Hrxn_total_exp = 14229000.00 J, Hrxn_total_pp = 14229000.00 J
n_molphen = 5.00e+02 mols, Hrxn_total_exp = 158100

In [6]:
# Constant Input Parameters
Ea = [128, 90, 100, 142] # Activation Energy
A = [2.34e10, 2.0e10, 2.1e10, 1.8e10] # Pre-exponential Factor
Hrxn = [-100, -100, -100, -100] # Heat of Reaction   

# Assign the values in the Ea list to identifiable variable names
Ea_pp_oo, Ea_pp_op, Ea_pp_pp, Ea_cp = Ea  # Activation Energy for each reaction type
A_pp_oo, A_pp_op, A_pp_pp, A_cp = A # Pre-exponential Factor for each reaction type

R = 8.314 # Universal Gas Constant

print(Ea_pp_oo, Ea_pp_op, Ea_pp_pp, Ea_cp)

In [ ]:
## Defining the model

## Identifying the inputs: 
    ## Temperature (T), preexponential factor (A), activation energy (Ea), time, t

## Defining the ranges: 
T = [300, 400, 500, 600, 700, 800, 900, 1000] # Kelvin
A = [1e-10, 1e-9, 1e-8, 1e-7] # 1/s
Ea_pp_o = [90] # kJ/mol